In [2]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import string
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference


In [3]:
df = pd.read_excel('/Users/Jay/Downloads/supermarket_sales.xlsx')
df_new = df[["Gender", "Product line", "Total"]]
df_new

,Gender,Product line,Total
0,Female,Health and beauty,548.9715
1,Female,Electronic accessories,80.2200
2,Male,Home and lifestyle,340.5255
3,Male,Health and beauty,489.0480
4,Male,Sports and travel,634.3785
...,...,...,...
995,Male,Health and beauty,42.3675
996,Female,Home and lifestyle,1022.4900
997,Male,Food and beverages,33.4320
998,Male,Home and lifestyle,69.1110


In [4]:
df_pivot = df_new.pivot_table(index = "Gender", 
                              columns = "Product line", 
                              values = "Total", 
                              aggfunc = "sum").round().astype(int)
df_pivot

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102,30437,33171,18561,30037,28575
Male,27236,23868,22974,30633,23825,26548


In [5]:
df_pivot.to_excel("/Users/Jay/Downloads/month_sales.xlsx", sheet_name = "Sales")

In [6]:
def automate_excel(file_name):
    df = pd.read_excel(f'/Users/Jay/Downloads/{file_name}')
    df_new = df[["Gender", "Product line", "Total"]]
    df_pivot = df_new.pivot_table(index = "Gender", 
                              columns = "Product line", 
                              values = "Total", 
                              aggfunc = "sum").round().astype(int)
    month_extention = f'{file_name}'.split("_")[1]
    
    df_pivot.to_excel(f'/Users/Jay/Downloads/report_{month_extention}', sheet_name = "Report", startrow = 4 )
    
    wb = load_workbook(f'/Users/Jay/Downloads/report_{month_extention}')
    
    sheet = wb["Report"]
    
    min_column = wb.active.min_column
    max_column = wb.active.max_column
    max_row = wb.active.max_row
    min_row = wb.active.min_row
    
    
    barchart = BarChart()
    data = Reference(sheet,
                    min_col = min_column +1,
                    max_col = max_column,
                    max_row = max_row,
                    min_row = min_row)
    
    categories = Reference(sheet,
                          min_col = min_column,
                          max_col = min_column,
                          max_row = max_row,
                          min_row = min_row+1)
    
    barchart.add_data(data, titles_from_data = True)
    barchart.set_categories(categories)
    barchart.style = 2
    sheet.add_chart(barchart, "B12")
    barchart.title = "Sales by Product Line"
    
    
    alphabet = list(string.ascii_uppercase)
    excel_alphabet = alphabet[0:max_column]
    
    for i in excel_alphabet:
        if i != "A":
            sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1} : {i}{max_row})' 
            sheet[f'{i}{max_row+1}'].style = "Currency"
            
    sheet[f'{excel_alphabet[0]}{max_row+1}'] = "Total"
    
    
    month_name = month_extention.split(".")[0]
    
    sheet["A1"]:sheet["G1"] = "Sales Report"
    sheet["A2"] = month_name.title()
    sheet["A1"].font = Font('Arial', bold = True, size = 20)
    sheet["A2"].font = Font('Arial', bold =True, size = 10)
    
    wb.save(f'/Users/Jay/Downloads/report_{month_extention}')
    return
    

In [7]:
automate_excel("sales_2022.xlsx")